<a href="https://colab.research.google.com/github/RamyaRamasubramaniyan/PysparkProjects/blob/main/Spark_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install pyspark

In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [4]:
spark

In [9]:
df = spark.read.csv('/content/Mall_Customers.csv', header=True)

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/content/Mall_Customers.csv.